Importing required packages

In [1]:
from functools import reduce

import numpy as np

from missionbio.h5.data.reader import H5Reader
from missionbio.h5.constants import (
    BARCODE,
    ID,
)

import pandas as pd

Read in the .h5 file gotten from the Genome Editing pipeline

In [2]:
with H5Reader("../data/Extended-Data-Fig6.ge.h5") as h5_file:
    ge_assay = h5_file.read('ge_dna_variants')
    fes = h5_file.read_raw_counts('final_edit_status')

Now, the scope is increase as we are only interested in two of our amplicons: <br>
#AMPL1023271 which is BRACA2
#AMPL1023282 which is PARP1 

In [3]:
#selecting the target amplicon
target_amp = 'AMPL1023271'
amplicon_columns = ge_assay.col_attrs['amplicon'] == target_amp
#keep only target amplicon columns (variants)
ge_assay.select_columns(amplicon_columns)

In [4]:
#selecting the target amplicon
target_amp = 'AMPL1023271'
amplicon_columns = ge_assay.col_attrs['amplicon'] == target_amp
#keep only target amplicon columns (variants)
ge_assay.select_columns(amplicon_columns)

In [5]:
wt_columns = ge_assay.col_attrs['is_ref'].astype(bool)
no_call_columns = ge_assay.col_attrs['is_nocall'].astype(bool)

snv_columns = ge_assay.col_attrs['is_snv']
indel_columns = np.logical_not(
    reduce(np.logical_or, [snv_columns, wt_columns, no_call_columns])
)
#the sum of wt_columns, no_call_columns, snv_columns and indel_columns is equal to the shape of each, 
#because this is the only things a cell can be called as
sum(wt_columns) + sum(no_call_columns) + sum(snv_columns) + sum(indel_columns)

#make a boolean matrix Cell x Variant. If the column is a variant (a SNV or indel) or in other word is NOT 
#a wt_column or a no_call_column, the entire column (all cells) is True.
variant_columns = np.tile(np.logical_or(snv_columns, indel_columns), (ge_assay.shape[0], 1))

#similarly make a boolean matrix Cell x Variant or Wt. Same principle, but wt is included as an acceptable criteria to be True
variant_and_wt_columns = np.tile(
    reduce(np.logical_or, (snv_columns, indel_columns, wt_columns)), (ge_assay.shape[0], 1)
)

Perform filtering on using QC criteria on the genotype quality, allele frequency and read depth for the P53 relevant loci

In [6]:
gt = ge_assay.layers['NGT']
gq = ge_assay.layers['GQ']
dp = ge_assay.layers['DP']
af = np.true_divide(ge_assay.layers['AD'], dp)

gq_filter = np.logical_and(gq < 30, variant_columns)
af_filter = np.logical_and(af < 0.1, variant_columns)
dp_filter = np.logical_and(dp < 10, variant_and_wt_columns)

combined_filters = reduce(np.logical_or, [gq_filter, af_filter, dp_filter])

gt_filtered = gt.copy()
gt_filtered[combined_filters] = 0
gt_filtered = np.nan_to_num(gt_filtered)

#BRACA2 subset
print(gq_filter.sum())
print(af_filter.sum())
print(dp_filter.sum())
print(combined_filters.sum())

6
15
113
132


/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/97270245.py:4: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)


Identify cells (rows) that lack a genotype in this amplicon

In [7]:
amp_var_wt_cols = np.logical_and(variant_and_wt_columns[0,:], amplicon_columns)

no_call_cells = (gt_filtered[:, np.logical_and(no_call_columns, amplicon_columns)] > 1).any(axis=1)
allele1_pass_filter = (gt_filtered[:,amp_var_wt_cols] == 1).any(axis=1)
allele2_pass_filter = (gt_filtered[:,amp_var_wt_cols] == 2).any(axis=1)
both_alleles_pass_filter = (gt_filtered[:, amp_var_wt_cols] == 3).any(axis=1)

no_call_cells = np.logical_or(no_call_cells, np.logical_not(np.logical_or(both_alleles_pass_filter, np.logical_and(allele1_pass_filter, allele2_pass_filter))))

Identify the variant calls on the two alleles across all cells (rows)

In [8]:
output = {}

amp_variant_ids = ge_assay.col_attrs[ID][amp_var_wt_cols]

for index, barcode in enumerate(ge_assay.row_attrs[BARCODE]):
    output[barcode] = dict(allele1=list(), allele2=list())
    if no_call_cells[index]:
        output[barcode]['allele1'].append(None)
        output[barcode]['allele2'].append(None)
    else:
        cell_gt_row = gt_filtered[index, amp_var_wt_cols]
        output[barcode]['allele1'].extend(amp_variant_ids[cell_gt_row==1])
        output[barcode]['allele2'].extend(amp_variant_ids[cell_gt_row==2])
        output[barcode]['allele1'].extend(amp_variant_ids[cell_gt_row==3])
        output[barcode]['allele2'].extend(amp_variant_ids[cell_gt_row==3])

In [9]:
#make the output into a pandas dataframe
output_df = pd.DataFrame(output).T

#export it as a .txt file
output_df.to_csv("../Extended-Data-Fig6/Extended-Data-Fig6_BRCA2.txt")

__now the same is done for the PARP1 amplicon__

In [10]:
with H5Reader("../data/Extended-Data-Fig6.ge.h5") as h5_file:
    ge_assay = h5_file.read('ge_dna_variants')
    fes = h5_file.read_raw_counts('final_edit_status')

#selecting the target amplicon for PARP1
target_amp = 'AMPL1023282'
amplicon_columns = ge_assay.col_attrs['amplicon'] == target_amp
#keep only target amplicon columns (variants)
ge_assay.select_columns(amplicon_columns)

#selecting the target amplicon for PARP1
target_amp = 'AMPL1023282'
amplicon_columns = ge_assay.col_attrs['amplicon'] == target_amp
#keep only target amplicon columns (variants)
ge_assay.select_columns(amplicon_columns)

wt_columns = ge_assay.col_attrs['is_ref'].astype(bool)
no_call_columns = ge_assay.col_attrs['is_nocall'].astype(bool)

snv_columns = ge_assay.col_attrs['is_snv']
indel_columns = np.logical_not(
    reduce(np.logical_or, [snv_columns, wt_columns, no_call_columns])
)
#the sum of wt_columns, no_call_columns, snv_columns and indel_columns is equal to the shape of each, 
#because this is the only things a cell can be called as
sum(wt_columns) + sum(no_call_columns) + sum(snv_columns) + sum(indel_columns)

#make a boolean matrix Cell x Variant. If the column is a variant (a SNV or indel) or in other word is NOT 
#a wt_column or a no_call_column, the entire column (all cells) is True.
variant_columns = np.tile(np.logical_or(snv_columns, indel_columns), (ge_assay.shape[0], 1))

#similarly make a boolean matrix Cell x Variant or Wt. Same principle, but wt is included as an acceptable criteria to be True
variant_and_wt_columns = np.tile(
    reduce(np.logical_or, (snv_columns, indel_columns, wt_columns)), (ge_assay.shape[0], 1)
)

gt = ge_assay.layers['NGT']
gq = ge_assay.layers['GQ']
dp = ge_assay.layers['DP']
af = np.true_divide(ge_assay.layers['AD'], dp)

gq_filter = np.logical_and(gq < 30, variant_columns)
af_filter = np.logical_and(af < 0.1, variant_columns)
dp_filter = np.logical_and(dp < 10, variant_and_wt_columns)

combined_filters = reduce(np.logical_or, [gq_filter, af_filter, dp_filter])

gt_filtered = gt.copy()
gt_filtered[combined_filters] = 0
gt_filtered = np.nan_to_num(gt_filtered)

print(gq_filter.sum())
print(af_filter.sum())
print(dp_filter.sum())
print(combined_filters.sum())

variant_and_wt_columns[0,:].shape
amp_var_wt_cols = np.logical_and(variant_and_wt_columns[0,:], amplicon_columns)

no_call_cells = (gt_filtered[:, np.logical_and(no_call_columns, amplicon_columns)] > 1).any(axis=1)
allele1_pass_filter = (gt_filtered[:,amp_var_wt_cols] == 1).any(axis=1)
allele2_pass_filter = (gt_filtered[:,amp_var_wt_cols] == 2).any(axis=1)
both_alleles_pass_filter = (gt_filtered[:, amp_var_wt_cols] == 3).any(axis=1)

no_call_cells = np.logical_or(no_call_cells, np.logical_not(np.logical_or(both_alleles_pass_filter, np.logical_and(allele1_pass_filter, allele2_pass_filter))))

output = {}

amp_variant_ids = ge_assay.col_attrs[ID][amp_var_wt_cols]

for index, barcode in enumerate(ge_assay.row_attrs[BARCODE]):
    output[barcode] = dict(allele1=list(), allele2=list())
    if no_call_cells[index]:
        output[barcode]['allele1'].append(None)
        output[barcode]['allele2'].append(None)
    else:
        cell_gt_row = gt_filtered[index, amp_var_wt_cols]
        output[barcode]['allele1'].extend(amp_variant_ids[cell_gt_row==1])
        output[barcode]['allele2'].extend(amp_variant_ids[cell_gt_row==2])
        output[barcode]['allele1'].extend(amp_variant_ids[cell_gt_row==3])
        output[barcode]['allele2'].extend(amp_variant_ids[cell_gt_row==3])

#make the output into a pandas dataframe
output_df = pd.DataFrame(output).T

#export it as a .txt file
output_df.to_csv("../Extended-Data-Fig6/Extended-Data-Fig6_PARP1.txt")

/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/2181139907.py:40: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)


1
169
99
268


__The following is to perform variant callings on all other amplicons included in the Tapestri run__<br>
The BRCA2 loci are seperated into three amplicons<br>
#AMPL1023271 (BRACA2 used for the other analysis)<br>
#AMPL1036549, <br>
#AMPL1036550 and <br>
#AMPL1036551<br>

For other possible off targets of interest.<br>
AMPL1023258 (chr1:34177936-34177937)<br>
AMPL1023259 (chr1:43452511-43452512)<br>
AMPL1023260 (chr1:44221167-44221168)<br>
AMPL1023261 (chr1:157283072-157283073)<br>
AMPL1023263 (chr1:247123143-247123144)<br>
AMPL1023264 (chr2:15308246-15308247)<br>
AMPL1023265 (chr3:37822832-37822833)<br>
AMPL1023266 (chr4:55429420-55429421)<br>
AMPL1023267 (chr5:53092008-53092009)<br>
AMPL1023268 (chr7:138076435-138076436)<br>
AMPL1023269 (chr7:149749879-149749880)<br>
AMPL1023270 (chr12:26494136-26494137)<br>
AMPL1023273 (chr15:43771262-43771263)<br>
AMPL1023274 (chr16:11120721-11120722)<br>
AMPL1023275 (chr17:9173898-9173899)<br>
AMPL1023276 (chr16:11120721-11120722)<br>
AMPL1023277 (chrX:41201253-41201254)<br>

In [11]:
import numpy as np
import pandas as pd
from functools import reduce

# List of target indices
target_indices = ['AMPL1023258', 'AMPL1023259', 'AMPL1023260', 'AMPL1023261', 'AMPL1023263', 'AMPL1023264',
                 'AMPL1023265', 'AMPL1023266', 'AMPL1023267', 'AMPL1023268', 'AMPL1023269', 'AMPL1023270',
                 'AMPL1023273', 'AMPL1023274', 'AMPL1023275', 'AMPL1023276', 'AMPL1023277',
                  'AMPL1023271', 'AMPL1036549', 'AMPL1036550', 'AMPL1036551']  # Replace with your actual indices


for target_amp in target_indices:
    with H5Reader("../data/Extended-Data-Fig6.ge.h5") as h5_file:
        ge_assay = h5_file.read('ge_dna_variants')
        fes = h5_file.read_raw_counts('final_edit_status')

    # Selecting the target amplicon
    amplicon_columns = ge_assay.col_attrs['amplicon'] == target_amp
    ge_assay.select_columns(amplicon_columns)

    # Selecting the target amplicon
    amplicon_columns = ge_assay.col_attrs['amplicon'] == target_amp
    ge_assay.select_columns(amplicon_columns)

    wt_columns = ge_assay.col_attrs['is_ref'].astype(bool)
    no_call_columns = ge_assay.col_attrs['is_nocall'].astype(bool)

    snv_columns = ge_assay.col_attrs['is_snv']
    indel_columns = np.logical_not(
        reduce(np.logical_or, [snv_columns, wt_columns, no_call_columns])
    )

    variant_columns = np.tile(np.logical_or(snv_columns, indel_columns), (ge_assay.shape[0], 1))
    variant_and_wt_columns = np.tile(
        reduce(np.logical_or, (snv_columns, indel_columns, wt_columns)), (ge_assay.shape[0], 1)
    )

    gt = ge_assay.layers['NGT']
    gq = ge_assay.layers['GQ']
    dp = ge_assay.layers['DP']
    af = np.true_divide(ge_assay.layers['AD'], dp)

    gq_filter = np.logical_and(gq < 30, variant_columns)
    af_filter = np.logical_and(af < 0.1, variant_columns)
    dp_filter = np.logical_and(dp < 10, variant_and_wt_columns)

    combined_filters = reduce(np.logical_or, [gq_filter, af_filter, dp_filter])

    gt_filtered = gt.copy()
    gt_filtered[combined_filters] = 0
    gt_filtered = np.nan_to_num(gt_filtered)

    amp_var_wt_cols = np.logical_and(variant_and_wt_columns[0, :], amplicon_columns)

    no_call_cells = (gt_filtered[:, np.logical_and(no_call_columns, amplicon_columns)] > 1).any(axis=1)
    allele1_pass_filter = (gt_filtered[:, amp_var_wt_cols] == 1).any(axis=1)
    allele2_pass_filter = (gt_filtered[:, amp_var_wt_cols] == 2).any(axis=1)
    both_alleles_pass_filter = (gt_filtered[:, amp_var_wt_cols] == 3).any(axis=1)

    no_call_cells = np.logical_or(
        no_call_cells,
        np.logical_not(np.logical_or(both_alleles_pass_filter, np.logical_and(allele1_pass_filter, allele2_pass_filter)))
    )

    output = {}
    amp_variant_ids = ge_assay.col_attrs[ID][amp_var_wt_cols]

    for index, barcode in enumerate(ge_assay.row_attrs[BARCODE]):
        output[barcode] = dict(allele1=list(), allele2=list())
        if no_call_cells[index]:
            output[barcode]['allele1'].append(None)
            output[barcode]['allele2'].append(None)
        else:
            cell_gt_row = gt_filtered[index, amp_var_wt_cols]
            output[barcode]['allele1'].extend(amp_variant_ids[cell_gt_row == 1])
            output[barcode]['allele2'].extend(amp_variant_ids[cell_gt_row == 2])
            output[barcode]['allele1'].extend(amp_variant_ids[cell_gt_row == 3])
            output[barcode]['allele2'].extend(amp_variant_ids[cell_gt_row == 3])

    # Make the output into a pandas dataframe
    output_df = pd.DataFrame(output).T

    # Export it as a .txt file with a dynamic filename
    output_df.to_csv(f"../Extended-Data-Fig6/off_target_amplicons/Extended-Data-Fig6_{target_amp}.txt")

/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/1661684646.py:41: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)
/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/1661684646.py:41: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)
/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/1661684646.py:41: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)
/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/1661684646.py:41: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)
/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/1661684646.py:41: RuntimeWarning: invalid value encountered in divide
  af = np.true_divide(ge_assay.layers['AD'], dp)
/var/folders/l2/gkfp0gmj30j2vx86rfhgw0p00000gn/T/ipykernel_89988/1661684646.py:4